In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from datetime import datetime, date, timedelta
import numpy as np
import random
import pickle
import statistics
import calendar
%matplotlib inline
from matplotlib import colors
from matplotlib.ticker import PercentFormatter





# Creating the list of tickers, cleaning it and fixing new token names

In [2]:
f = open("binance_list.txt", "r")
lines = f.readlines()
new_ticker_list = []
for lines in lines:
    #print(i)
    j = lines.split('\t')
    j = j[2].split('/')[0]
    if j != 'BTC':
        new_ticker_list.append(j)
new_ticker_list[12] = "MIOTA"
new_ticker_list[37] = "HOLO"
new_ticker_list[86] = "BQX"
new_ticker_list[99] = "YOYOW"

In [3]:
#This function, borrowed directly from the Cryptocompare API tutorial, extracts the data for the price of a coin, in BTC
#The function goes back until the starting the date for the experiment, using nb_of_days.
def get_data_spec(coin,nb_of_days,start_date):
    #We need to convert our start_date into a UNIX time stamp.
    start_date = int(time.mktime(start_date.timetuple()))
    """ Query the API for 2000 units historical price data starting from "start_date". """
    url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym=BTC&limit={}&e=Binance".format(coin,nb_of_days)
    r = requests.get(url)
    ipdata = r.json()
    df = pd.DataFrame(ipdata['Data'])
    return df

In [4]:
#This function creates a list of DFs with the ticker and the historical price data for every desired coin.
#It takes as the starting and end date of the experiment, in order to calculate the nb of days.
#To deal with my UTC - 3 clock, I just need to make the conversion to get to 0:00 UTC.
def create_holder(start,end):
    end = end - timedelta(hours = 3)
    start = start - timedelta(hours = 3)
    delta = end - start
    nb_of_days = delta.days
    holder = []
    for coin in new_ticker_list:
        objeto = [coin, get_data_spec(coin,nb_of_days,start)]
        holder.append(objeto)
    return(holder,nb_of_days)

In [5]:
#This function creates the matrix that tells us whether the coin existed on Binance at the specific interval
#We chose to define 0 if the coin was not there and 1 if it was.

def construct_matrix(interval):
    nb_months = int(np.floor(days/interval))
    verification_matrix = pd.DataFrame(np.zeros((len(new_ticker_list), nb_months+1)), dtype = np.int8)
    verification_matrix[0] = new_ticker_list
    tol = 10**-9
    for i in range(len(holder)):
        linha = 0
        for j in range(1,nb_months+1):
            if len(holder[i][1]) == 0:
                price = 0
            else:
                price = holder[i][1].loc[linha][0] 
            if price <= 0 + tol:
                verification_matrix.iloc[i,j] = 0
            else:
                verification_matrix.iloc[i,j] = 1
            linha += interval
    names = ['Coin',]
    count = 0
    for j in range(nb_months):
        nome = 'Interval' + '_' + str(count)
        names.append(nome)
        count +=1
    verification_matrix.columns = names
    return(verification_matrix,nb_months)

In [6]:
def calculate_rf(start_date, end_date):
    #I need to calculate the return of a buy-hold and sell at the end strategy for BTC.
    #I actually only need the price in USD in the start date and in the end date.
    start_date = start_date - timedelta(hours = 3)
    start_date = int(time.mktime(start_date.timetuple()))
    url ="https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USDT&limit=1&toTs={}&e=Binance".format(start_date)
    r = requests.get(url)
    ipdata = r.json()
    df = pd.DataFrame(ipdata['Data'])
    start_price = df.iloc[1,3]
    end_date = end_date - timedelta(hours = 3)
    end_date = int(time.mktime(end_date.timetuple()))
    url ="https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USDT&limit=1&toTs={}&e=Binance".format(end_date)
    r = requests.get(url)
    ipdata = r.json()
    df = pd.DataFrame(ipdata['Data'])
    end_price = df.iloc[1,3]
    rf = (end_price - start_price) / start_price 
    return(rf, start_price, end_price)

#This function calculates the ex-post sharpe ratio, as defined in 1994.
def calculate_sharpe(rf,return_vector):
    #transform return_vector in excess_returns
    excess_returns = []
    excess_returns[:] = [a - rf for a in return_vector]
    sharpe_medium = np.mean(excess_returns)/ np.std(excess_returns)
    sharpe_median = statistics.median(excess_returns) / np.std(excess_returns)
    return(sharpe_medium, sharpe_median)

    

# Creating the monkeys and their methods


In [7]:
class Monkey:
    def __init__(self,seed,attitude,funds,interval):
        self.seed = seed
        random.seed(seed)
        self.attitude = attitude
        #Since calculate_rf gives us the starting and ending price for BTC, our funds will be in USD
        self.initial_funds = funds
        self.funds = funds/start_price #These are the funds CONVERTED to BTC
        self.final_funds = 0
        self.increase = 0
        self.interval = interval
        self.initial_wallet()
        
    def initial_wallet(self):
        possible_coins = verification_matrix[verification_matrix.Interval_0 == 1]['Coin'].tolist()
        chosen_coins = random.sample(possible_coins,5)
        value_invested = self.funds/5
        self.wallet = {}
        a = 1
        for i in chosen_coins:
            self.wallet[a] = {}
            price = holder[new_ticker_list.index(i)][1].loc[0][3]
            quantity = value_invested / price 
            quantity = quantity * (1 - 10**-4) #Discounts the BinanceFee
            self.wallet[a]['Coin'] = i
            self.wallet[a]['Price_paid'] = price
            self.wallet[a]['Quantity'] = quantity
            self.wallet[a]['Profit'] = 0
            a += 1

    
    
    def evaluate_wallet(self,month):
        wallet = self.wallet
        for coin in wallet:
            coin_information = wallet[coin]
            new_price = holder[new_ticker_list.index(coin_information['Coin'])][1].loc[self.interval*month][3]
            quantity = coin_information['Quantity']
            #To evaluate the profit of a coin, you need to simulate how its sale would be, which implies a fee payment.
            coin_profit = new_price*quantity*(1-10**-4) -coin_information['Price_paid']*quantity 
            self.wallet[coin]['Profit'] = coin_profit
            
    def update_wallet(self,old_coin_index,month):
        wallet = self.wallet
        #wallet[old_coin_index] will give us the coin to be replaced
        current_month = verification_matrix.columns[1 + month]
        possible_coins = verification_matrix[verification_matrix[current_month] == 1]['Coin'].tolist()
        possible_coins.remove(wallet[old_coin_index]['Coin'])
        chosen_coin = random.choice(possible_coins)
        #Now that you have chosen the coin to replace the previous one...
        #You need to update your wallet.
        newcoin_price = holder[new_ticker_list.index(chosen_coin)][1].loc[self.interval*month][3]
        oldcoin_price = holder[new_ticker_list.index(wallet[old_coin_index]['Coin'])][1].loc[self.interval*month][3]
        #First, sell the old coin. 
        
        quantity = wallet[old_coin_index]['Quantity'] * oldcoin_price * (1-10**-4)
       
        #Now, we have the quantity in BTC.
        
        #Next, we use that to buy the new coin
        
        quantity = (quantity/newcoin_price) * (1-10**4)
        
        wallet[old_coin_index]['Coin'] = chosen_coin
        wallet[old_coin_index]['Price_paid'] = newcoin_price
        wallet[old_coin_index]['Quantity'] = quantity
        wallet[old_coin_index]['Profit'] = 0
    def final_evaluation(self):
        wallet = self.wallet
        for i in wallet:
            coin_information = wallet[i]
            coin = coin_information['Coin']
            final_price = holder[new_ticker_list.index(coin)][1].loc[self.interval*nb_months][3]
            quantity = coin_information['Quantity']
            profit = final_price*quantity*(1-10**-4) -coin_information['Price_paid']*quantity 
            coin_information['Profit'] = profit
            #The only time you ACTUALLY pocket the profit is when you sell all holdings to BTC, then go back to USD
            #So basically, you need to calculate how much BTC is there in your wallet...
            self.final_funds += final_price*quantity*(1-10**-4)
        self.final_funds = self.final_funds*(1-10**-4)*end_price #This converts it back to USD.
        self.increase = (self.final_funds - self.initial_funds) / self.initial_funds #This is the return in USD, as it should be.
            
    def monthly_sale(self,month):
        self.evaluate_wallet(month)
        wallet = self.wallet
        attitude = self.attitude
        if month == nb_months:
            self.final_evaluation()
        else:
            profit_list = []
            for coin in wallet:
                profit_list.append(wallet[coin]['Profit'])
            if self.attitude != 'old':
                if attitude != 'crazy_random':
                    worst = min(profit_list)
                    best = max(profit_list)
                    if attitude == 'crazy_good':
                        old_coin_index = profit_list.index(best) +1
                    if attitude == 'crazy_bad':
                        old_coin_index = profit_list.index(worst) + 1
                    if attitude == 'crazy_alternating':
                        if month % 2 == 0:
                            old_coin_index = profit_list.index(best) + 1
                        else:
                            old_coin_index = profit_list.index(worst) + 1
                else:
                    old_coin_index = profit_list.index(random.choice(profit_list)) + 1
                self.update_wallet(old_coin_index,month)  

# Algorithm start


In [ ]:
random.seed(5)
seeds_list = random.sample(range(10**9), 20)
interval_list = []
count = 5
while count != 50:
    interval_list.append(count)
    count += 5
#We have defined a number of possible intervals for the strategies to be implemented.
#Then you need to define starting dates and end dates
d0 = datetime(2017,11,1)
d1 = datetime(2019,6,12)
#Then, we generate the holder for all prices
v = create_holder(d0,d1)
holder = v[0]
days = v[1]




#Next, we calculate the return rate of holding BTC for the period
info = calculate_rf(d0,d1)
rf = info[0]
start_price = info[1]
end_price = info[2]







In [ ]:
#Calculate the old monkey's sharpe ratio, since it does not depend on interval length
returns_list = []
big_list = {}
for seed in seeds_list:
    general = construct_matrix(30)
    verification_matrix = general[0]
    nb_months = general[1]
    monkey = Monkey(seed,'old',7500,30)
    for month in range(1,nb_months+1):
        monkey.monthly_sale(month)
    returns_list.append(monkey.increase)
#Now, calculate its sharpe ratio
old_sharpe = calculate_sharpe(rf,returns_list)


In [ ]:
#Calculate the rest of the sharpe ratios for every possible combination of interval and strategy.
sharpe_dict = {}
sharpe_dict['old'] = old_sharpe
monkey_list = []
for i in interval_list:
    general = construct_matrix(i)
    verification_matrix = general[0]
    nb_months = general[1]
    for j in ['crazy_good','crazy_bad','crazy_random','crazy_alternating']:
        returns_list = []
        for seed in seeds_list:
            monkey = Monkey(seed,j,7500,i)
            for month in range(1,nb_months+1):
                monkey.monthly_sale(month)
            monkey_list.append(monkey)
            returns_list.append(monkey.increase)
        sharpe = calculate_sharpe(rf,returns_list)   
        sharpe_dict[j + '-' + str(i)] = sharpe

            

sharpe_dict
